In [1]:
import os
if os.getcwd().endswith('notebooks'):
    os.chdir(os.path.dirname(os.getcwd()))
    
from src.datasets import stellar_data
from src.models import vanilla_stellar
import torch
import numpy as np
import argparse

d:\dlls-pro\dlls-pro-venv\Lib\site-packages\torch_geometric\typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: [WinError 127] Nie można odnaleźć określonej procedury
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
GRAPH_DATASET_FILENAME = 'stellar_graph_dataset.pt'

In [3]:
torch.manual_seed(42)
sample_idx_permutations = []
n_folds = 5
n_train = 100
for i in range(n_folds):
    idx_perm = torch.randperm(125).tolist()
    sample_idx_permutations.append({"train" : idx_perm[:n_train], "valid" : idx_perm[n_train:]})

In [7]:
cfg_reduced = argparse.Namespace(**{
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'input_dim': 40,
    'hid_dim': 128, # originally 128
    'num_classes': 14,
    'lr': 1e-3,
})

In [8]:
batch_size = 1
n_epochs = 20

In [9]:
valid_accs = np.zeros(n_folds)
for k in range(n_folds):
    print(f"Fold {k} of {n_folds}")
    train_dataloader = stellar_data.StellarDataloader(
        filename=GRAPH_DATASET_FILENAME,
        batch_size=batch_size,
        shuffle=True,
        graphs_idx=sample_idx_permutations[k]["train"],
        test=False
    )
    valid_dataloader = stellar_data.StellarDataloader(
        filename=GRAPH_DATASET_FILENAME,
        batch_size=batch_size,
        shuffle=False,
        graphs_idx=sample_idx_permutations[k]["valid"],
        test=False
    )

    vanilla_stellar_reduced = vanilla_stellar.VanillaStellarReduced(cfg_reduced)
    valid_acc = vanilla_stellar_reduced.train(
        train_loader=train_dataloader,
        valid_loader=valid_dataloader,
        epochs=n_epochs,
        return_valid_acc=True)
    valid_accs[k] = valid_acc

Fold 0 of 5


Validation - epoch 19: 100%|██████████| 25/25 [00:00<00:00, 76.12it/s, Loss=0.232, Accuracy=0.926] 


Fold 1 of 5


Validation - epoch 19: 100%|██████████| 25/25 [00:00<00:00, 116.34it/s, Loss=0.32, Accuracy=0.894] 


Fold 2 of 5


Validation - epoch 19: 100%|██████████| 25/25 [00:00<00:00, 113.84it/s, Loss=0.186, Accuracy=0.935]


Fold 3 of 5


Validation - epoch 19: 100%|██████████| 25/25 [00:00<00:00, 84.57it/s, Loss=0.331, Accuracy=0.885]


Fold 4 of 5


Validation - epoch 19: 100%|██████████| 25/25 [00:00<00:00, 50.52it/s, Loss=0.256, Accuracy=0.918]


In [10]:
print(f"Mean validation accuracy: {valid_accs.mean()} +/- {valid_accs.std()}")

Mean validation accuracy: 0.9114213490486144 +/- 0.019208156609532963
